In [7]:
import time
from IPython.display import clear_output
import subprocess
import psutil
from tqdm import tqdm

interval = 0.5
# 获取初始的网络 I/O 统计信息
prev_net_io = psutil.net_io_counters()
prev_bytes_sent = prev_net_io.bytes_sent
prev_bytes_recv = prev_net_io.bytes_recv

prev_disk_io = psutil.disk_io_counters()
prev_disk_write = prev_disk_io.write_bytes
prev_disk_read = prev_disk_io.read_bytes

In [ ]:
while True:
    # 清空输出区域
    clear_output(wait=True)
    # 使用 nvidia-smi 命令获取显卡状态
    result = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE)
    # 获取 CPU 占用率
    cpu_usage = psutil.cpu_percent(interval=interval)
    # 获取内存占用率
    memory_info = psutil.virtual_memory()
    memory_usage = memory_info.percent

    # 获取当前的网络 I/O 统计信息
    current_net_io = psutil.net_io_counters()
    current_bytes_sent = current_net_io.bytes_sent
    current_bytes_recv = current_net_io.bytes_recv

    # 获取硬盘 I/O 信息
    current_disk_io = psutil.disk_io_counters()
    current_disk_write = current_disk_io.write_bytes
    current_disk_read = current_disk_io.read_bytes
    
    # 计算每秒的网络发送和接收字节数
    bytes_sent_per_sec = (current_bytes_sent - prev_bytes_sent) / interval
    bytes_recv_per_sec = (current_bytes_recv - prev_bytes_recv) / interval
    
    # 计算每秒的io发送和接收字节数
    disk_write = (current_disk_write - prev_disk_write) / interval
    disk_read = (current_disk_read - prev_disk_read) / interval
    
    # 更新上一次统计信息
    prev_bytes_sent = current_bytes_sent
    prev_bytes_recv = current_bytes_recv
    prev_disk_write = current_disk_write
    prev_disk_read = current_disk_read

    print(result.stdout.decode("utf-8"))
    print(f"CPU 占用率: {cpu_usage}%")
    print(f"每秒网络发送(MB): {bytes_sent_per_sec / 1024 ** 2}")
    print(f"每秒网络接收(MB): {bytes_recv_per_sec / 1024 ** 2}")
    # 获取所有进程
    all_processes = psutil.process_iter(["pid", "name", "memory_info"])
    # 存储符合条件的进程信息
    python_processes = []
    # 过滤出名称包含 'python' 或 'pt' 的进程
    for proc in all_processes:
        try:
            if (
                "milvus" in proc.info["name"].lower()
                or "pt_" in proc.info["name"].lower()
                or "python" in proc.info["name"].lower()
            ):
                name = proc.info["name"]
                pid = proc.info["pid"]
                memory_info = proc.info["memory_info"]
                memory_usage_mb = memory_info.rss / 1024 / 1024  # 以MB为单位
                python_processes.append((pid, name, memory_usage_mb))
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    # 按内存使用从高到低排序
    python_processes.sort(key=lambda x: x[2], reverse=True)

    # 打印排序后的进程信息
    for pid, name, memory_usage_mb in python_processes[:8]:
        print(f"进程号: {pid}, 进程名：{name}, 内存使用: {memory_usage_mb:.2f} MB")
    # 每隔一秒输出一次
    print(f"内存占用率: {memory_usage}%")
    print(f"每秒io写入(MB): {disk_write / 1024 ** 2}")
    print(f"每秒io读取(MB): {disk_read / 1024 ** 2}")
    time.sleep(interval)

Wed Dec 11 18:18:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      Off| 00000000:04:00.0 Off |                  N/A |
|  0%   30C    P8               25W / 300W|   7409MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--